In [1]:
!pip install numpy
!pip install pandas
!pip install decord
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117
!pip install pytorch-lightning
!pip install pytorchvideo
!pip install scikit-learn
!pip install scikit-multilearn
!pip install segmentation-models-pytorch
!pip install transformers
!pip install einops
!pip install tqdm
!pip install opencv-python
!pip install albumentations

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117


In [2]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader

import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore')

from einops import rearrange
from decord import VideoReader
from sklearn.metrics import f1_score
from segmentation_models_pytorch.losses import FocalLoss
from transformers import AutoModel, AutoImageProcessor, AutoConfig
from skmultilearn.model_selection import iterative_train_test_split
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorchvideo.transforms.transforms_factory import create_video_transform

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"device : {device}")

device : cuda


In [4]:
CFG = {
    'VIDEO_LENGTH':50, # 10프레임 * 5초
    'IMG_SIZE':128,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':4,
    'SEED':41
}

In [5]:
train_df = pd.read_csv('./data/train.csv')
train_df.head()

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,7
1,TRAIN_0001,./train/TRAIN_0001.mp4,7
2,TRAIN_0002,./train/TRAIN_0002.mp4,0
3,TRAIN_0003,./train/TRAIN_0003.mp4,0
4,TRAIN_0004,./train/TRAIN_0004.mp4,1


Crash + Ego-involved Class
0: Non-crash
1: ego involved
2: Non ego involved

In [6]:
# Non-crash
df0 = train_df[train_df['label'] == 0]

# ego involved
df1 = train_df[train_df['label'] <= 6]
df1 = df1[df1['label'] >= 1]
df1['label'] = 1

# Non ego involved
df2 = train_df[train_df['label'] >= 7]
df2['label'] = 2

In [7]:
crash_ego_df = pd.concat([df0, df1, df2])

In [8]:
crash_ego_df = crash_ego_df.sort_index()
crash_ego_df

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,2
1,TRAIN_0001,./train/TRAIN_0001.mp4,2
2,TRAIN_0002,./train/TRAIN_0002.mp4,0
3,TRAIN_0003,./train/TRAIN_0003.mp4,0
4,TRAIN_0004,./train/TRAIN_0004.mp4,1
...,...,...,...
2693,TRAIN_2693,./train/TRAIN_2693.mp4,1
2694,TRAIN_2694,./train/TRAIN_2694.mp4,1
2695,TRAIN_2695,./train/TRAIN_2695.mp4,0
2696,TRAIN_2696,./train/TRAIN_2696.mp4,0


In [9]:
drop_non_crash = train_df[train_df['label'] >= 1]
drop_non_crash

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,7
1,TRAIN_0001,./train/TRAIN_0001.mp4,7
4,TRAIN_0004,./train/TRAIN_0004.mp4,1
6,TRAIN_0006,./train/TRAIN_0006.mp4,3
7,TRAIN_0007,./train/TRAIN_0007.mp4,7
...,...,...,...
2685,TRAIN_2685,./train/TRAIN_2685.mp4,8
2689,TRAIN_2689,./train/TRAIN_2689.mp4,1
2692,TRAIN_2692,./train/TRAIN_2692.mp4,7
2693,TRAIN_2693,./train/TRAIN_2693.mp4,3


Timing Class
0: Night
1: Day

In [10]:
# Night
df0 = drop_non_crash[drop_non_crash['label'] % 2 == 0]
df0['label'] = 0

# Day
df1 = drop_non_crash[drop_non_crash['label'] % 2 == 1]
df1['label'] = 1

In [11]:
timing_df = pd.concat([df0, df1])
timing_df = timing_df.sort_index()
timing_df

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,1
1,TRAIN_0001,./train/TRAIN_0001.mp4,1
4,TRAIN_0004,./train/TRAIN_0004.mp4,1
6,TRAIN_0006,./train/TRAIN_0006.mp4,1
7,TRAIN_0007,./train/TRAIN_0007.mp4,1
...,...,...,...
2685,TRAIN_2685,./train/TRAIN_2685.mp4,0
2689,TRAIN_2689,./train/TRAIN_2689.mp4,1
2692,TRAIN_2692,./train/TRAIN_2692.mp4,1
2693,TRAIN_2693,./train/TRAIN_2693.mp4,1


Weather Class
0: Normal
1: Snowy
2: Rainy

In [12]:
def define_weather(x):
    if 1<=x<=2 or 7<=x<=8:
        return 0
    elif 3<=x<=4 or 9<=x<=10:
        return 1
    else:
        return 2

In [13]:
weather_df = pd.DataFrame(drop_non_crash)
weather_df['label'] = drop_non_crash['label'].apply(define_weather)
weather_df

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,0
1,TRAIN_0001,./train/TRAIN_0001.mp4,0
4,TRAIN_0004,./train/TRAIN_0004.mp4,0
6,TRAIN_0006,./train/TRAIN_0006.mp4,1
7,TRAIN_0007,./train/TRAIN_0007.mp4,0
...,...,...,...
2685,TRAIN_2685,./train/TRAIN_2685.mp4,0
2689,TRAIN_2689,./train/TRAIN_2689.mp4,0
2692,TRAIN_2692,./train/TRAIN_2692.mp4,0
2693,TRAIN_2693,./train/TRAIN_2693.mp4,1


In [14]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

먼저, crash_ego_df 부터 학습

In [15]:
crash_ego_df.head()

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,2
1,TRAIN_0001,./train/TRAIN_0001.mp4,2
2,TRAIN_0002,./train/TRAIN_0002.mp4,0
3,TRAIN_0003,./train/TRAIN_0003.mp4,0
4,TRAIN_0004,./train/TRAIN_0004.mp4,1


In [16]:
crash_ego_df['label'].value_counts()

0    1783
1     491
2     424
Name: label, dtype: int64

In [17]:
crash_ego_df_train, crash_ego_df_val, _, _ = train_test_split(crash_ego_df, crash_ego_df['label'], test_size=0.2, random_state=CFG['SEED'])

In [18]:
def video_augmentation(vid, tfms):
    seed = random.randint(0,99999)
    aug_vid = []
    for x in vid:
        random.seed(seed)
        aug_vid.append((tfms(image = np.asarray(x)))['image'])
    return torch.from_numpy(np.stack(aug_vid))

In [19]:
tfms = A.Compose([
    A.Resize(width=CFG['IMG_SIZE'], height=CFG['IMG_SIZE']),
    A.HorizontalFlip(p=0.5),
    A.Normalize()
], p=1)

In [20]:
class Dataset(Dataset):
    def __init__(self, video_path_list, label_list, tfms):
        self.video_path_list = video_path_list
        self.label_list = label_list
        self.tfms = tfms

    def __getitem__(self, index):
        frames = self.get_video(self.video_path_list[index])

        if self.label_list is not None:
            label = self.label_list[index]
            return frames, label
        else:
            return frames

    def __len__(self):
        return len(self.video_path_list)

    def get_video(self, path):
        frames = []
        path = './data' + path[1:]
        cap = cv2.VideoCapture(path)
        for _ in range(CFG['VIDEO_LENGTH']):
            _, img = cap.read()
            frames.append(img)
        frames = video_augmentation(frames, tfms=self.tfms)
        return torch.FloatTensor(np.array(frames)).permute(3, 0, 1, 2)

In [21]:
train_dataset = Dataset(crash_ego_df_train['video_path'].values, crash_ego_df_train['label'].values, tfms=tfms)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = Dataset(crash_ego_df_val['video_path'].values, crash_ego_df_val['label'].values, tfms=tfms)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [22]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=13):
        super(BaseModel, self).__init__()
        self.feature_extract = nn.Sequential(
            nn.Conv3d(3, 8, (1, 3, 3)),
            nn.ReLU(),
            nn.BatchNorm3d(8),
            nn.MaxPool3d(2),
            nn.Conv3d(8, 32, (1, 2, 2)),
            nn.ReLU(),
            nn.BatchNorm3d(32),
            nn.MaxPool3d(2),
            nn.Conv3d(32, 64, (1, 2, 2)),
            nn.ReLU(),
            nn.BatchNorm3d(64),
            nn.MaxPool3d(2),
            nn.Conv3d(64, 128, (1, 2, 2)),
            nn.ReLU(),
            nn.BatchNorm3d(128),
            nn.MaxPool3d((3, 7, 7)),
        )
        self.classifier = nn.Linear(1024, num_classes)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.feature_extract(x)
        x = x.view(batch_size, -1)
        x = self.classifier(x)
        return x

In [23]:
class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, weight=self.weight, reduction=self.reduction)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1-pt)**self.gamma * ce_loss).mean()
        return focal_loss

In [24]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = FocalLoss().to(device)

    best_val_score = 0
    best_model = None

    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            output = model(videos)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')

        if scheduler is not None:
            scheduler.step(_val_score)

        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model

    return best_model

In [25]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []

    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)

            logit = model(videos)

            loss = criterion(logit, labels)

            val_loss.append(loss.item())

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()

        _val_loss = np.mean(val_loss)

    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

In [26]:
first_model = BaseModel()
first_model.eval()
optimizer = torch.optim.Adam(params = first_model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(first_model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/540 [00:00<?, ?it/s]

  0%|          | 0/135 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.37002] Val Loss : [0.16487] Val F1 : [0.62607]


  0%|          | 0/540 [00:00<?, ?it/s]

  0%|          | 0/135 [00:00<?, ?it/s]

...

  0%|          | 0/135 [00:00<?, ?it/s]

Epoch [9], Train Loss : [0.07012] Val Loss : [0.08898] Val F1 : [0.74147]


  0%|          | 0/540 [00:00<?, ?it/s]

  0%|          | 0/135 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.06911] Val Loss : [0.08068] Val F1 : [0.77297]


In [27]:
test = pd.read_csv('./data/test.csv')

In [28]:
test_dataset = Dataset(test['video_path'].values, None, tfms=tfms)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [29]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)

            logit = model(videos)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [30]:
first_preds = inference(first_model, test_loader, device)

  0%|          | 0/450 [00:00<?, ?it/s]

In [31]:
first_preds

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 1,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 2,
 0,
 0,
 0,
 2,
 1,
 0,
 1,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 2,
 0,
 1,
 2,


In [35]:
timing_df.head()

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,1
1,TRAIN_0001,./train/TRAIN_0001.mp4,1
4,TRAIN_0004,./train/TRAIN_0004.mp4,1
6,TRAIN_0006,./train/TRAIN_0006.mp4,1
7,TRAIN_0007,./train/TRAIN_0007.mp4,1


In [36]:
timing_df['label'].value_counts()

1    808
0    107
Name: label, dtype: int64

In [37]:
timing_df_train, timing_df_val, _, _ = train_test_split(timing_df, timing_df['label'], test_size=0.2, random_state=CFG['SEED'])

In [38]:
train_dataset = Dataset(timing_df_train['video_path'].values, timing_df_train['label'].values, tfms=tfms)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = Dataset(timing_df_val['video_path'].values, timing_df_val['label'].values, tfms=tfms)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [39]:
second_model = BaseModel()
second_model.eval()
optimizer = torch.optim.Adam(params = second_model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(second_model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/183 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.13369] Val Loss : [0.01711] Val F1 : [0.86101]


  0%|          | 0/183 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

...

  0%|          | 0/46 [00:00<?, ?it/s]

Epoch [9], Train Loss : [0.00117] Val Loss : [0.01934] Val F1 : [0.87568]


  0%|          | 0/183 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.00149] Val Loss : [0.02895] Val F1 : [0.92278]
Epoch 00010: reducing learning rate of group 0 to 7.5000e-05.


In [40]:
second_preds = inference(second_model, test_loader, device)

  0%|          | 0/450 [00:00<?, ?it/s]

In [41]:
weather_df.head()

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,0
1,TRAIN_0001,./train/TRAIN_0001.mp4,0
4,TRAIN_0004,./train/TRAIN_0004.mp4,0
6,TRAIN_0006,./train/TRAIN_0006.mp4,1
7,TRAIN_0007,./train/TRAIN_0007.mp4,0


In [42]:
weather_df['label'].value_counts()

0    716
1    129
2     70
Name: label, dtype: int64

In [43]:
weather_df_train, weather_df_val, _, _ = train_test_split(weather_df, weather_df['label'], test_size=0.2, random_state=CFG['SEED'])

In [44]:
train_dataset = Dataset(weather_df_train['video_path'].values, weather_df_train['label'].values, tfms=tfms)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = Dataset(weather_df_val['video_path'].values, weather_df_val['label'].values, tfms=tfms)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [45]:
third_model = BaseModel()
third_model.eval()
optimizer = torch.optim.Adam(params = second_model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(second_model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/183 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.40957] Val Loss : [0.20437] Val F1 : [0.47690]


  0%|          | 0/183 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

...

  0%|          | 0/46 [00:00<?, ?it/s]

Epoch [9], Train Loss : [0.04188] Val Loss : [0.13788] Val F1 : [0.50303]


  0%|          | 0/183 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.03975] Val Loss : [0.11844] Val F1 : [0.56261]


In [46]:
third_preds = inference(first_model, test_loader, device)

  0%|          | 0/450 [00:00<?, ?it/s]

In [47]:
first_preds

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 1,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 2,
 0,
 0,
 0,
 2,
 1,
 0,
 1,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 2,
 0,
 1,
 2,


In [48]:
second_preds

[0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,


In [49]:
third_preds

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 2,
 2,
 0,
 0,
 2,
 1,
 0,
 1,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 2,


In [52]:
len(first_preds)

1800

In [53]:
len(second_preds)

1800

In [54]:
len(third_preds)

1800

In [55]:
final_preds = []
for i in range(1800):
    
    # crash + ego 가 0 일때,
    if first_preds[i] == 0:
        final_preds.append(0)
        
    # crash + ego 가 1 일때,
    elif first_preds[i] == 1:
        # Night
        if second_preds[i] == 0:
            if third_preds[i] == 0:
                final_preds.append(2)
            elif third_preds[i] == 1:
                final_preds.append(4)
            else:
                final_preds.append(6)
        # Day
        else:
            if third_preds[i] == 0:
                final_preds.append(1)
            elif third_preds[i] == 1:
                final_preds.append(3)
            else:
                final_preds.append(5)
                
    # crash + ego 가 2 일때,
    else:
        # Night
        if second_preds[i] == 0:
            if third_preds[i] == 0:
                final_preds.append(8)
            elif third_preds[i] == 1:
                final_preds.append(10)
            else:
                final_preds.append(12)
        # Day
        else:
            if third_preds[i] == 0:
                final_preds.append(7)
            elif third_preds[i] == 1:
                final_preds.append(9)
            else:
                final_preds.append(11)

In [56]:
final_preds

[0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 9,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 9,
 0,
 11,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 11,
 3,
 0,
 0,
 0,
 0,
 11,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 11,
 3,
 0,
 11,
 0,
 11,
 0,
 11,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 11,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 11,
 11,
 3,
 0,
 0,
 7,
 0,
 11,
 0,
 0,
 1,
 0,
 0,
 11,
 0,
 0,
 11,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 5,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 11,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 11,
 0,
 0,
 0,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 11,
 0,
 0,
 3,
 7,
 0,
 0,
 11,
 0,
 0,
 0,
 0,
 0,
 11,
 11,
 0,
 3,
 11,
 0,
 0,
 0,
 11,
 3,
 0,
 3,
 0,
 3,
 11,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 11,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 11,
 12,
 0,
 9,
 0,
 3,
 3,
 0,
 0,
 0,
 0,
 11,
 0,
 0,
 11,
 0,
 11,
 11,
 0,

In [61]:
len(final_preds)

1800

In [57]:
submit = pd.read_csv('./data/sample_submission.csv')

In [59]:
submit['label'] = final_preds
submit.head()

,sample_id,label
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0


In [60]:
submit.to_csv('./submission_0227.csv', index=False)